In [26]:
import math
import multiprocessing as mg
import multiprocessing.pool
# import pys2let as ps
import random
import string
import itertools
import os

import jax
jax.config.update("jax_enable_x64", True)
import s2fft
import healpy as hp
import numpy as np
import s2wav
import s2wav
import matplotlib.pyplot as plt
%matplotlib inline 
import skyclean
from skyclean import CMB_data

In [27]:
def mw_alm_2_hp_alm(MW_alm2, lmax):
    # Initialize the 1D hp_alm array with the appropriate size
    hp_alm1 = np.zeros(hp.Alm.getsize(lmax), dtype=np.complex128)
        
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            index = hp.Alm.getidx(lmax, l, abs(m))
            if m < 0:
                hp_alm1[index] = (-1)**m * np.conj(MW_alm2[l, lmax + m])
            else:
                hp_alm1[index] = MW_alm2[l, lmax + m]

    return hp_alm1

# Doubleworker

## Loaded mw wavelet coefficient map

## Covert to mw alm space

## add zero to the mw alms  (Is it correct? or should I add zeros to the hp alm's and then convert to mw alm's)

## Convert to from mw alm to mw map      (hp)


def Single_Map_doubleworker(MW_Pix_Map):
    '''
    Input: MW_Pix_Map: list of mw maps at different scales 
    Each pixel map is a wavelet pixel map of shape (1, Lmax, 2*Lmax-1) (MW sampling, McEwen & Wiaux)
    It is the output of s2wav.analysis
    (Scale: 0, size (1, 4, 7))

    Process:
    1. Covert MW Pixel Map to MW alm space using s2fft.forward

    2. Add zero to the mw alms  (Is it correct? or should I add zeros to the hp alm's and then convert to mw alm's)
    by adding zeros to the MW alm's we are increasing the resolution of the map
    Double the rows of the mw alms, since, the number of rows represents the L (level of detail)
    
    3. Convert mw alm to mw map 
    
    '''
    print("original Pixel Map size", MW_Pix_Map.shape)
    MW_alm = s2fft.forward(MW_Pix_Map, L = MW_Pix_Map.shape[1])
    print("original alm size", MW_alm.shape)
    
   
    # print("Scale:",i,"original alm size", MW_alm[i].shape)
    padded_alm = np.zeros((MW_alm.shape[0]*2,MW_alm.shape[1]*2))
    # stored_wavelet_coeffs_alm_doubled.append(skyclean.double_resolution(stored_wavelet_coeffs_alm[i]))
    padded_alm[:MW_alm.shape[0], :MW_alm.shape[1]] = MW_alm
    print("padded alm size", padded_alm.shape)
    MW_alm_doubled = padded_alm
    
    MW_Pix_Map_doubled = s2fft.inverse(MW_alm_doubled, L = MW_alm_doubled.shape[0])
    print("Scale:","doubled map size", MW_Pix_Map_doubled.shape)

    return MW_Pix_Map_doubled

## Loaded mw wavelet coefficient map
stored_wavelet_coeffs_pix = [np.load(f"../convolution/wavelet_coefficient/wav_30_{i}.npy", allow_pickle=True) for i in range(12)]
stored_scaling_coeffs_pix = np.load("../convolution/scaling_coefficient/scal_30.npy")


# print(stored_wavelet_coeffs_pix[0].shape)
stored_wavelet_coeffs_pix = stored_wavelet_coeffs_pix[:3]

wavelet_MW_Pix_Map_doubled = Single_Map_doubleworker(stored_wavelet_coeffs_pix[0])

# display(wavelet_MW_Pix_Map_doubled[0])
# Why oen dimension is reduced?
# Is it the spin?
print(wavelet_MW_Pix_Map_doubled.shape)


original Pixel Map size (1, 4, 7)
original alm size (4, 7)
padded alm size (8, 14)
Scale: doubled map size (8, 15)
(8, 15)


In [40]:
def smoothworker(MW_Map1, MW_Map2, smoothing_lmax, scale_fwhm):
    # Get the real part of the map
    map1 = np.real(MW_Map1)
    map2 = np.real(MW_Map2)
    # Coveriance matrix
    R = np.multiply(map1,map2) + 0.j #Add back in zero imaginary part
    print("R", R.shape)

    R_MW_alm = s2fft.forward(R, L = smoothing_lmax)
    print("R_MW_alm", R_MW_alm.shape)

    R_hp_alm = mw_alm_2_hp_alm(R_MW_alm,smoothing_lmax-1)
    # R_hp_alm1 = mw_alm_2_hp_alm(R_MW_alm,smoothing_lmax-1)
    print("R_hp_alm", R_hp_alm.shape)

    gauss_smooth = hp.gauss_beam(scale_fwhm,lmax=smoothing_lmax-1)
    hp.almxfl(R_hp_alm,gauss_smooth,inplace=True) #Multiply by gaussian beam
    # print(gauss_smooth.shape)
    # print("R_hp_alm", R_hp_alm.shape)

    MW_alm_beam_convolved = np.zeros(R_MW_alm.shape, dtype=np.complex128)

    for i in range(R_MW_alm.shape[1]):
        MW_alm_beam_convolved[:, i] = R_MW_alm[:, i] * gauss_smooth
    # CMB_data_30_pix_beam_deconvolved.shape

    # print(R_hp_alm == R_hp_alm1)
    return R_hp_alm, MW_alm_beam_convolved


npix = hp.nside2npix(1<<(int(0.5*wavelet_MW_Pix_Map_doubled.shape[0])-1).bit_length())
# (int(0.5*scale_lmax)-1).bit_length() calculates the number of bits necessary to represent the integer int(0.5*scale_lmax)-1 in binary.
# 1 << (int(0.5*scale_lmax)-1).bit_length() performs a bitwise left shift, essentially calculating 2^(number of bits).
scale_fwhm = 4.0 * math.sqrt(1200 / npix)
smoothworker(wavelet_MW_Pix_Map_doubled, wavelet_MW_Pix_Map_doubled,wavelet_MW_Pix_Map_doubled.shape[0], scale_fwhm)

hp_alm_1, MW_alm = smoothworker(wavelet_MW_Pix_Map_doubled, wavelet_MW_Pix_Map_doubled,wavelet_MW_Pix_Map_doubled.shape[0], scale_fwhm)

R (8, 15)
R_MW_alm (8, 15)
R_hp_alm (36,)
R (8, 15)
R_MW_alm (8, 15)
R_hp_alm (36,)


In [41]:
hp_alm_mw =  mw_alm_2_hp_alm(MW_alm, wavelet_MW_Pix_Map_doubled.shape[0]-1)
display(hp_alm_1)
display(hp_alm_mw)

array([ 2.42214339e-040+3.40237899e-057j,
       -4.48611920e-049+3.55088446e-065j,
       -4.34207483e-064-4.79158275e-081j,
        3.65686041e-088+1.90546914e-104j,
        2.28946809e-119+2.65340099e-135j,
       -1.56812203e-159-1.18453325e-174j,
       -1.48336698e-206-2.23969144e-222j,
        5.13630232e-261-1.91042402e-276j,
       -1.81066533e-048-8.25506863e-065j,
        4.80335685e-065-4.28400754e-080j,
        6.85205340e-088+7.47681292e-104j,
       -1.25055167e-119+6.49658658e-135j,
       -5.75114723e-159-1.54463325e-174j,
       -2.29659457e-206-2.56662597e-221j,
        5.21747260e-276+9.33454136e-277j,
        2.62130082e-065-4.17203505e-080j,
        3.59319463e-088+1.10131227e-103j,
       -1.08450252e-119+4.13280982e-135j,
       -8.81746376e-159+2.50255719e-174j,
        7.31054191e-206-1.44914783e-221j,
       -2.07593507e-275-1.34962725e-276j,
       -1.27038614e-088-7.14138760e-104j,
       -2.47079766e-119+1.66374767e-135j,
        6.23488842e-159+1.56471526

array([ 2.42214339e-040+3.40237899e-057j,
       -4.48611920e-049+3.55088446e-065j,
       -4.34207483e-064-4.79158275e-081j,
        3.65686041e-088+1.90546914e-104j,
        2.28946809e-119+2.65340099e-135j,
       -1.56812203e-159-1.18453325e-174j,
       -1.48336698e-206-2.23969144e-222j,
        5.13630232e-261-1.91042402e-276j,
       -1.81066533e-048-8.25506863e-065j,
        4.80335685e-065-4.28400754e-080j,
        6.85205340e-088+7.47681292e-104j,
       -1.25055167e-119+6.49658658e-135j,
       -5.75114723e-159-1.54463325e-174j,
       -2.29659457e-206-2.56662597e-221j,
        5.21747260e-276+9.33454136e-277j,
        2.62130082e-065-4.17203505e-080j,
        3.59319463e-088+1.10131227e-103j,
       -1.08450252e-119+4.13280982e-135j,
       -8.81746376e-159+2.50255719e-174j,
        7.31054191e-206-1.44914783e-221j,
       -2.07593507e-275-1.34962725e-276j,
       -1.27038614e-088-7.14138760e-104j,
       -2.47079766e-119+1.66374767e-135j,
        6.23488842e-159+1.56471526

In [35]:
hp_alm_1

array([ 2.42214339e-040+3.40237899e-057j,
       -4.48611920e-049+3.55088446e-065j,
       -4.34207483e-064-4.79158275e-081j,
        3.65686041e-088+1.90546914e-104j,
        2.28946809e-119+2.65340099e-135j,
       -1.56812203e-159-1.18453325e-174j,
       -1.48336698e-206-2.23969144e-222j,
        5.13630232e-261-1.91042402e-276j,
       -1.81066533e-048-8.25506863e-065j,
        4.80335685e-065-4.28400754e-080j,
        6.85205340e-088+7.47681292e-104j,
       -1.25055167e-119+6.49658658e-135j,
       -5.75114723e-159-1.54463325e-174j,
       -2.29659457e-206-2.56662597e-221j,
        5.21747260e-276+9.33454136e-277j,
        2.62130082e-065-4.17203505e-080j,
        3.59319463e-088+1.10131227e-103j,
       -1.08450252e-119+4.13280982e-135j,
       -8.81746376e-159+2.50255719e-174j,
        7.31054191e-206-1.44914783e-221j,
       -2.07593507e-275-1.34962725e-276j,
       -1.27038614e-088-7.14138760e-104j,
       -2.47079766e-119+1.66374767e-135j,
        6.23488842e-159+1.56471526

array([ 2.42214339e-040+0.j, -4.48611920e-049+0.j, -4.34207483e-064+0.j,
        3.65686041e-088+0.j,  2.28946809e-119+0.j, -1.56812203e-159+0.j,
       -1.48336698e-206+0.j,  5.13630232e-261+0.j, -1.81066533e-048+0.j,
        4.80335685e-065+0.j,  6.85205340e-088+0.j, -1.25055167e-119+0.j,
       -5.75114723e-159+0.j, -2.29659457e-206+0.j,  5.21747260e-276+0.j,
        2.62130082e-065+0.j,  3.59319463e-088+0.j, -1.08450252e-119+0.j,
       -8.81746376e-159+0.j,  7.31054191e-206+0.j, -2.07593507e-275+0.j,
       -1.27038614e-088+0.j, -2.47079766e-119+0.j,  6.23488842e-159+0.j,
        6.54095343e-206+0.j,  2.51140957e-276+0.j,  3.37119413e-119+0.j,
       -7.79971632e-159+0.j, -5.74249357e-206+0.j, -1.56199846e-275+0.j,
       -4.15171382e-158+0.j,  9.62858469e-206+0.j, -3.27397254e-275+0.j,
        2.80718920e-205+0.j,  3.78847581e-276+0.j, -1.39141904e-275+0.j])

In [31]:
def original_smoothworker(i): #(j,n,map_index1,map_index2,smoothing_lmax,scale_fwhm) [map_index2<=map_index1]
    print ("Smoothing independent covariance element", i[2], ",", i[3])

    #Map loading within sub-process
    if i[0] >= 0: #Wavelet scales
        wav_fits1 = wav_fits_root[i[2]] + '_' + wavparam_code + str(i[0]) + '_n' + str(i[1]+1) + '_double.npy'
        wav_fits2 = wav_fits_root[i[3]] + '_' + wavparam_code + str(i[0]) + '_n' + str(i[1]+1) + '_double.npy'
    else: #Scaling function
        wav_fits1 = scal_fits[i[2]][:-4] + '_double.npy'
        wav_fits2 = scal_fits[i[3]][:-4] + '_double.npy'
    map1 = np.real(np.load(wav_fits1,mmap_mode='r')) #Throw away zero imaginary part
    map2 = np.real(np.load(wav_fits2,mmap_mode='r'))
    R = np.multiply(map1,map2) + 0.j #Add back in zero imaginary part
    del map1,map2

    alms = ps.lm2lm_hp(ps.map2alm_mw(R,i[4],spin),i[4]) #No pixwin correct. with MW - calc alms to smooth - come out in MW order - so converted to HPX order
    del R
    gausssmooth = hp.gauss_beam(i[5],lmax=i[4]-1)
    hp.almxfl(alms,gausssmooth,inplace=True) #Multiply by gaussian beam

    print "Synthesising smoothed covariance map for element", i[2], ",", i[3]
    Rsmooth = np.real(ps.alm2map_mw(ps.lm_hp2lm(alms,i[4]),i[4],spin)) #Throw away zero imaginary part - input alm's in MW order
    del alms
    
    #SAVE smoothed covariance
    if i[0] >= 0: #Wavelet scales
        R_fits = wav_outfits_root + '_' + wavparam_code + str(i[0]) + '_n' + str(i[1]+1) + '_Rsmooth' + str(i[2]) + str(i[3]) +'.npy'
    else: #Scaling function
        R_fits = scal_outfits[:-4] + '_Rsmooth' + str(i[2]) + str(i[3]) +'.npy'
    np.save(R_fits,Rsmooth)
    del Rsmooth
    
    return 0


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (3490765726.py, line 21)

In [ ]:
a = np.array([[1,2],[3,4]])
b = np.array([[5,6],[7,8]])
# display(a)
# display(b)
# display(np.multiply(a,b))

In [ ]:
display(np.multiply(wavelet_MW_Pix_Map_doubled[0],wavelet_MW_Pix_Map_doubled[1]))

In [ ]:
def ILC(MW_Pix_Map):

    
    Current_Wavelet_Map = MW_Pix_Map
    
    # Define the size of the smoothing beam 
    # 1200 pixels, size of the sphere
    nsamp = 1200.0
    lmax_at_scale_j = Current_Wavelet_Map.shape[0]
    npix = hp.nside2npix(1<<(int(0.5*lmax_at_scale_j)-1).bit_length())
    # (int(0.5*scale_lmax)-1).bit_length() calculates the number of bits necessary to represent the integer int(0.5*scale_lmax)-1 in binary.
    # 1 << (int(0.5*scale_lmax)-1).bit_length() performs a bitwise left shift, essentially calculating 2^(number of bits).
    scale_fwhm = 4.0 * math.sqrt(nsamp / npix)
    # for high resolution maps, it is still the same number pixels sampled by the actual range is smaller.
    # the beam will become very narrow.
    print(lmax_at_scale_j,npix, scale_fwhm)





    


    

# ILC(wavelet_MW_Pix_Map_doubled)

# print(wavelet_MW_Pix_Map_doubled.shape)

In [ ]:
def s2let_ilc(mapsextra): #mapsextra = (j,n)
    print "Running S2LET ILC on wavelet scale", mapsextra[0], "/", jmax, "direction", mapsextra[1]+1, "/", ndir, "\n"

    if mapsextra[0] >= 0: #Wavelet scales
        scale_lmax = wav_bandlims[mapsextra[0]]
    else: #Scaling function
        scale_lmax = int(scal_bandlims)
    smoothing_lmax = 2*(scale_lmax-1)+1

    #Doubling lmax for input maps with zero-padding
    #Serial version
    '''mapsdouble = np.zeros((nrows,ps.mw_size(smoothing_lmax)),dtype=np.complex128) #Pre-allocate array
    for i in xrange(nrows):
        mapsdouble[i,:] = doubleworker((mapsextra[0][i],mapsextra[1],smoothing_lmax,mapsextra[2]))'''
    #Parallel version
    mapsextra2 = [(mapsextra[0],mapsextra[1],i,scale_lmax,smoothing_lmax) for i in xrange(nmaps)]
    print "Forming pool"
    pool2 = mg.Pool(nprocess2)
    print "Farming out workers to run doubling function"
    double_output = pool2.map(doubleworker,mapsextra2)
    print "Have returned from doubling workers\n"
    pool2.close()
    pool2.join()
    del pool2

    #Calculate scale_fwhm for smoothing kernel
    nsamp = 1200.
    npix = hp.nside2npix(1<<(int(0.5*scale_lmax)-1).bit_length()) #Equivalent no. HEALPIX pixels
    scale_fwhm = 4. * mh.sqrt(nsamp / npix)
    
    #TESTING larger covariance kernel
    scale_fwhm = 15.*scale_fwhm
    
    #Smooth covariance matrices
    #Serial version
    '''Rsmoothflat = np.zeros_like(Rflat) #Pre-allocate array
    for i in xrange(nindepelems):
        Rsmoothflat[i,:] = smoothworker((Rflat[i],smoothing_lmax,mapsextra[2],gausssmooth,mapsextra[1],mapsextra[3],i,mapsextra[4]))
    del Rflat'''
    #Parallel version
    nindepelems = int(nmaps*(nmaps+1)*.5) #No. indep. elements in symmetric covariance matrix
    Rextra = [None]*nindepelems
    k=0
    for i in xrange(nmaps):
        for j in xrange(i+1):
            Rextra[k] = (mapsextra[0],mapsextra[1],i,j,smoothing_lmax,scale_fwhm)
            k+=1
    print "Forming pool"
    pool3 = mg.Pool(nprocess3)
    print "Farming out workers to run smoothing function"
    R_output = pool3.map(smoothworker,Rextra)
    print "Have returned from smoothing workers\n"
    pool3.close()
    pool3.join()
    del pool3

    #Load R maps and form matrices
    print "Pre-allocating memory for complete covariance tensor\n"
    Rsmooth = np.zeros((ps.mw_size(smoothing_lmax),nmaps,nmaps),dtype=np.float64) #Pre-allocate array
    for i in xrange(nmaps):
        for j in xrange(i+1):
            if mapsextra[0] >= 0: #Wavelet scales
                R_fits = wav_outfits_root + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '_Rsmooth' + str(i + 9 - nmaps) + str(j + 9 - nmaps) +'.npy'
            else: #Scaling function
                R_fits = scal_outfits[:-4] + '_Rsmooth' + str(i + 9 - nmaps) + str(j + 9 - nmaps) +'.npy'
            Rsmooth[:,i,j] = np.load(R_fits)
            if i != j:
                Rsmooth[:,j,i] = Rsmooth[:,i,j]

    #Compute inverse covariance matrices
    print "Calculating inverse covariance matrices\n"
    Rinv = np.linalg.inv(Rsmooth) #Parallel vers. slower!?- LARGEST MEMORY COST: 2*9*9*(8000^2)*complex128=0.2TB
    del Rsmooth

    #Compute weights vectors (at each pixel)
    wknumer = np.sum(Rinv,axis=-1)
    del Rinv
    wkdenom = np.sum(wknumer,axis=-1)
    wk = wknumer / wkdenom[:,None]
    del wknumer,wkdenom

    #Map loading within sub-process
    mapsdouble = np.zeros((len(wk),len(wk[0])),dtype=np.float64) #Pre-allocate array
    for i in xrange(nmaps):
        if mapsextra[0] >= 0: #Wavelet scales
            wav_fits = wav_fits_root[i + 9 - nmaps] + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '_double.npy'
        else: #Scaling function
            wav_fits = scal_fits[i + 9 - nmaps][:-4] + '_double.npy'
        mapsdouble[:,i] = np.real(np.load(wav_fits,mmap_mode='r')) #Throw away zero imaginary part

    #Dot weights with maps (at each small pixel) - at double l(j)
    finalmap = np.sum(np.multiply(wk,mapsdouble),axis=-1) + 0.j #Add back in zero imaginary part
    del wk,mapsdouble
    
    #Downgrade resolution of MW maps
    print "Downgrading resolution of CMB wavelet map"
    finalmapalms = ps.lm2lm_hp(ps.map2alm_mw(finalmap,smoothing_lmax,spin),smoothing_lmax) #Come out in MW order - so converted to HPX order
    del finalmap
    if mapsextra[0] >= 0: #Wavelet scales
        alms_fname = wav_outfits_root + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '_alms.fits'
    else: #Scaling function
        alms_fname = scal_outfits[:-4] + '_alms.fits'
    hp.write_alm(alms_fname,finalmapalms,lmax=scale_lmax-1,mmax=scale_lmax-1)
    del finalmapalms
    finalmapalmstruncate = hp.read_alm(alms_fname)
    finalmaphalf = ps.alm2map_mw(ps.lm_hp2lm(finalmapalmstruncate,scale_lmax),scale_lmax,spin)
    del finalmapalmstruncate
    
    #Saving output map
    if mapsextra[0] >= 1: #0: #Wavelet scales
        wav_outfits = wav_outfits_root + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '.npy'
    elif mapsextra[0] == 0: #FOR NEW SCALING FUNC
        wav_outfits = wav_outfits_0 + '_' + wavparam_code + str(mapsextra[0]) + '_n' + str(mapsextra[1]+1) + '.npy'
    else: #Scaling function
        wav_outfits = scal_outfits[:-4] + '.npy'
    np.save(wav_outfits,finalmaphalf)
    del finalmaphalf

    return 0
